In [120]:

from transformers import AutoModelForCausalLM, AutoTokenizer, pipelines
from accelerate import init_empty_weights
import torch

In [122]:
import pandas as pd
dataset_path = "/home/fozle/Course Work/Y1S2/CISC 844/Projects/LLM-Predicate-Logic/dataset/data.csv"
df = pd.read_csv(dataset_path)

In [123]:
sample_size = 100  # Adjust based on how many examples you want to test
df_sample = df.sample(sample_size, random_state=42)

In [125]:
# Checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [126]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",  # or 3B version
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Device set to use cpu


In [127]:
#  load only one row from the dataset
row = df_sample.iloc[7]
row

id                                                            2893
logic                                                    predicate
rule_category                                            inference
rule                                      existential transitivity
problem                                              contradiction
query            Consider the following premises: There is at l...
answer                                                          no
Name: 2892, dtype: object

In [128]:

# user_prompt = row['query']
user_query = row['query']
print(user_query)
ground_truth = row['answer']
ground_truth

Consider the following premises: There is at least one x for which if x takes a plane, then x is listening to a song. For all x, if x listens to a song, then x is sleeping. Can we infer the following from them? Answer yes or no: For all x, the fact that x takes a plane does not imply that x is sleeping.


'no'

In [129]:
# Create an ICL prompt that includes example problems and their answers
system_prompt = """You are a predicate logic assistant specializing in formal logic analysis."""

In [110]:
user_prompt = f""" 
Your capabilities include:
1. Evaluating the validity of logical arguments using predicate logic
2. Identifying contradictions and fallacies
3. Translating natural language into predicate logic notation and vice versa
4. Assessing logical equivalence between statements
5. Performing logical operations such as negation, conjunction, disjunction, and implication

Here are some examples to guide your reasoning:

Example 1:
Problem: Consider the following premises: Neither Barbara is a writer nor Susan is playing a game.
Query: Can we infer the following from them? Answer yes or no: Barbara is not a writer and Susan is not playing a game.
Answer: yes

Example 2:
Problem: Consider the following premises: Neither Susan is drinking water nor Patricia is playing a game.
Query: Can we infer the following from them? Answer yes or no: Susan is not drinking water and Patricia is not playing a game.
Answer: yes

Example 3:
Problem: Consider the following premises: Neither it is raining nor Elisabeth is a server.
Query: Can we infer the following from them? Answer yes or no: It is not raining and Elisabeth is not a server.
Answer: yes

Example 4:
Problem: Consider the following premises: The claim that Susan is running and the claim that Charles is a janitor cannot both be true.
Query: Can we infer the following from them? Answer yes or no: Jessica is writing a letter.
Answer: no

Example 5:
Problem: Consider the following premises: The claim that it is late and the claim that Sarah is happy cannot both be true.
Query: Can we infer the following from them? Answer yes or no: It is not snowing.
Answer: no

You will be given a new logical problem and query. Respond ONLY with "yes" or "no" based on whether the conclusion logically follows from the premises.

{user_query}
Answer:
"""

In [130]:
user_prompt

' \nYour capabilities include:\n1. Evaluating the validity of logical arguments using predicate logic\n2. Identifying contradictions and fallacies\n3. Translating natural language into predicate logic notation and vice versa\n4. Assessing logical equivalence between statements\n5. Performing logical operations such as negation, conjunction, disjunction, and implication\n\nHere are some examples to guide your reasoning:\n\nExample 1:\nProblem: Consider the following premises: Neither Barbara is a writer nor Susan is playing a game.\nQuery: Can we infer the following from them? Answer yes or no: Barbara is not a writer and Susan is not playing a game.\nAnswer: yes\n\nExample 2:\nProblem: Consider the following premises: Neither Susan is drinking water nor Patricia is playing a game.\nQuery: Can we infer the following from them? Answer yes or no: Susan is not drinking water and Patricia is not playing a game.\nAnswer: yes\n\nExample 3:\nProblem: Consider the following premises: Neither it

In [131]:
system_prompt

'You are a predicate logic assistant specializing in formal logic analysis.'

In [136]:
 # Prepare messages for the model
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

outputs = pipe(
            messages,
            max_new_tokens=128,  # Shorter since we only need yes/no
            do_sample=False  # Deterministic response
        )
        
        
# Extract response content
response = outputs[0]["generated_text"][-1]["content"].strip().lower()
response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'yes'